In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftConfig, PeftModel
from peft.peft_model import PeftModelForSeq2SeqLM

# peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

tokenizer = T5Tokenizer.from_pretrained("/public/home/hongy/zljin/FT_flanT5xl/Pretrained/flan_t5_xl")
model = T5ForConditionalGeneration.from_pretrained("/public/home/hongy/zljin/FT_flanT5xl/Pretrained/flan_t5_xl")
# model = get_peft_model(model, peft_config)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: Required library version not found: libsbitsandbytes_cpu.so. Maybe you need to compile it from source?
CUDA SETUP: Defaulting to libbitsandbytes_cpu.so...


/public/home/hongy/miniconda3/envs/zljin_LLMs/lib/python3.9/site-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import torch
input_text = ["stsb sentence1: I'm going to have chicken tonight. sentence2: I will cook fish this evening.",
              "stsb sentence1: I will cook chicken this evening. sentence2: I'm going to have chicken tonight.",
              "stsb sentence1: I'm going to have chicken tonight. sentence2: I will cook fish this evening."]

tokenized_dict = tokenizer(input_text, return_tensors="pt")
input_ids = tokenized_dict['input_ids']
attention_mask = tokenized_dict['attention_mask']

bs_size = input_ids.shape[0]
print("batch_size:", bs_size)
decoder_start_token = 0
decoder_input_ids = torch.tensor([[decoder_start_token] for _ in range(bs_size)])
print(input_ids)
print(decoder_input_ids)

batch_size: 3
tensor([[   3,    7,   17,    7,  115, 7142,  536,   10,   27,   31,   51,  352,
           12,   43, 3832, 8988,    5, 7142,  357,   10,   27,   56, 3989, 2495,
           48, 2272,    5,    1],
        [   3,    7,   17,    7,  115, 7142,  536,   10,   27,   56, 3989, 3832,
           48, 2272,    5, 7142,  357,   10,   27,   31,   51,  352,   12,   43,
         3832, 8988,    5,    1],
        [   3,    7,   17,    7,  115, 7142,  536,   10,   27,   31,   51,  352,
           12,   43, 3832, 8988,    5, 7142,  357,   10,   27,   56, 3989, 2495,
           48, 2272,    5,    1]])
tensor([[0],
        [0],
        [0]])


In [3]:
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, use_cache=False, return_dict=True)

In [9]:
print(type(outputs))
logits = outputs[0].squeeze(1)
print(logits.shape)
res1 = logits[:, [4273, 150]]
print(res1)
res = torch.argmax(res1, dim=-1)
print(res)
res = res.unsqueeze(-1)
print(res.shape)
print(res)

<class 'transformers.modeling_outputs.Seq2SeqLMOutput'>
torch.Size([3, 32128])
tensor([[-11.3935,  -5.9171],
        [ -0.4979,  -6.3665],
        [-11.3935,  -5.9171]], grad_fn=<IndexBackward0>)
tensor([1, 0, 1])
torch.Size([3, 1])
tensor([[1],
        [0],
        [1]])


In [49]:
# generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, use_cache=True, num_beams=5, 
# 				length_penalty=0.6, max_new_tokens=1, repetition_penalty=2.0, decoder_start_token_id=decoder_start_token)

generated_ids = model.generate(input_ids=input_ids)

/public/home/hongy/miniconda3/envs/zljin_LLMs/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [40]:
no_ids = tokenizer("no", return_tensors='pt').input_ids
print(no_ids)

tensor([[150,   1]])


In [24]:
type(generated_ids)

generated_ids

tensor([[   0, 4273,    1]])

In [25]:
gen_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
gen_text

['yes']

In [ ]:
import os
def load_and_cache_examples(args, tokenizer, evaluate=False, max_q = None, max_a = None):
    if not evaluate:
        type_path = f'train{args.train_name}'
        logger.info("Loading training data from %s", type_path)
    else:
        type_path = 'dev'   # DIFF HERE
    dataset = TrainDataset(tokenizer, data_dir=args.data_dir, type_path=type_path, max_source_length=args.max_src_len, max_target_length=args.max_tgt_len)
    return dataset
eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True, max_q=max_q, max_a=max_a)
if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
    os.makedirs(eval_output_dir)

args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
# Note that DistributedSampler samples randomly
eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=eval_dataset.collate_fn)